### Data filter for testing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
network = pd.read_csv("D:/Pycharm/fulfillment_delay/jd_data/JD_data/JD_network_data.csv")
order = pd.read_csv("D:/Pycharm/fulfillment_delay/jd_data/pre_data/order_1p.csv")
order['order_date'] = pd.to_datetime(order['order_date'])
order['order_time'] = pd.to_datetime(order['order_time'])

In [3]:
fdc = 43
rdc = int(network[network['dc_ID'] == fdc]['region_ID'])
print(fdc, rdc)
order_fdc = order[order['dc_des'] == fdc]
order_fdc_fdc_rdc = order_fdc[order_fdc.dc_ori.isin([fdc, rdc])]
order_fdc_fdc_rdc.info()

43 2
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3792 entries, 152 to 273507
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_ID                    3792 non-null   object        
 1   user_ID                     3792 non-null   object        
 2   sku_ID                      3792 non-null   object        
 3   order_date                  3792 non-null   datetime64[ns]
 4   order_time                  3792 non-null   datetime64[ns]
 5   quantity                    3792 non-null   int64         
 6   type                        3792 non-null   int64         
 7   promise                     3792 non-null   object        
 8   original_unit_price         3792 non-null   float64       
 9   final_unit_price            3792 non-null   float64       
 10  direct_discount_per_unit    3792 non-null   float64       
 11  quantity_discount_per_unit  3792 non-null   flo

In [4]:
# duplicate orders with the same datetime
tmp_order_fdc_fdc_rdc = order_fdc_fdc_rdc.drop_duplicates(subset=['order_ID'], keep='first')
tmp_order_fdc_fdc_rdc['order_time'].value_counts()

2018-03-02 20:03:52    3
2018-03-01 11:34:33    2
2018-03-04 19:00:33    2
2018-03-08 10:39:08    2
2018-03-01 20:46:11    2
                      ..
2018-03-09 17:55:45    1
2018-03-09 18:08:53    1
2018-03-09 18:42:46    1
2018-03-09 18:58:46    1
2018-03-31 23:55:34    1
Name: order_time, Length: 3444, dtype: int64

In [5]:
# duplicate orders with the same datetime
time = tmp_order_fdc_fdc_rdc['order_time'].unique()
order_fdc_fdc_rdc = order_fdc_fdc_rdc.set_index(['order_ID', 'sku_ID'])
for t in time:
     tmp_co = tmp_order_fdc_fdc_rdc[tmp_order_fdc_fdc_rdc['order_time'] == t]
     tmp_clen = len(tmp_co.index)
     if tmp_clen > 1:
          ci = tmp_co.index.tolist()
          for i in range(1, tmp_clen):
               cr_date = tmp_co.loc[ci[i]]['order_time']
               while cr_date in time:
                    a = int(np.random.choice(range(1, 61)))
                    cr_date += datetime.timedelta(seconds=a)
               oi = tmp_order_fdc_fdc_rdc.loc[ci[i], 'order_ID']
               order_fdc_fdc_rdc.loc[order_fdc_fdc_rdc.index.get_level_values('order_ID') == oi, 'order_time'] = cr_date
               tmp_order_fdc_fdc_rdc.loc[ci[i], 'order_time'] = cr_date
               time = tmp_order_fdc_fdc_rdc['order_time'].unique()
tmp_order_fdc_fdc_rdc['order_time'].value_counts()


2018-03-01 00:03:05    1
2018-03-23 15:52:44    1
2018-03-23 14:25:55    1
2018-03-23 14:28:38    1
2018-03-23 14:39:31    1
                      ..
2018-03-09 14:44:01    1
2018-03-09 14:46:21    1
2018-03-09 14:49:48    1
2018-03-09 15:05:14    1
2018-03-31 23:55:34    1
Name: order_time, Length: 3471, dtype: int64

In [6]:
order_fdc_fdc_rdc.reset_index(inplace=True)
tmp = order_fdc_fdc_rdc.drop_duplicates(subset=['order_ID'], keep='first')
tmp['order_time'].value_counts()

2018-03-01 00:03:05    1
2018-03-23 15:52:44    1
2018-03-23 14:25:55    1
2018-03-23 14:28:38    1
2018-03-23 14:39:31    1
                      ..
2018-03-09 14:44:01    1
2018-03-09 14:46:21    1
2018-03-09 14:49:48    1
2018-03-09 15:05:14    1
2018-03-31 23:55:34    1
Name: order_time, Length: 3471, dtype: int64

In [7]:
order_fdc_fdc_rdc['order_time'].value_counts()

2018-03-25 12:38:58    4
2018-03-22 20:37:10    4
2018-03-26 12:35:40    4
2018-03-13 21:31:35    4
2018-03-02 14:20:22    3
                      ..
2018-03-10 12:44:51    1
2018-03-10 13:05:00    1
2018-03-10 13:05:26    1
2018-03-10 13:32:20    1
2018-03-31 23:55:34    1
Name: order_time, Length: 3471, dtype: int64

In [8]:
order_fdc_fdc_rdc.to_csv('./pre_data/order_fdc_fdc_rdc.csv', index=False)

### Date

In [9]:
order = order_fdc_fdc_rdc
temp = order.drop_duplicates(subset=['order_ID'], keep='first')

In [10]:
order = order.set_index('order_time')
order = order.sort_index()
temp = temp.set_index('order_time')
temp = temp.sort_index()

In [11]:
# tmp: Selling in each day
temp["count"] = 1
temp_fdc = temp[temp['dc_ori'] == fdc]
temp_rdc = temp[temp['dc_ori'] == rdc]
temp_fdc_one_day = temp_fdc.resample('D').sum()
temp_rdc_one_day = temp_rdc.resample('D').sum()
day_list = temp_rdc_one_day.index.format()

fdc_day_num = temp_fdc_one_day['count'].tolist()
rdc_day_num = temp_rdc_one_day['count'].tolist()
print(day_list, len(day_list))
print(fdc_day_num, sum(fdc_day_num))
print(rdc_day_num, sum(rdc_day_num))

['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08', '2018-03-09', '2018-03-10', '2018-03-11', '2018-03-12', '2018-03-13', '2018-03-14', '2018-03-15', '2018-03-16', '2018-03-17', '2018-03-18', '2018-03-19', '2018-03-20', '2018-03-21', '2018-03-22', '2018-03-23', '2018-03-24', '2018-03-25', '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29', '2018-03-30', '2018-03-31'] 31
[189, 40, 62, 55, 54, 54, 76, 102, 62, 36, 42, 69, 93, 73, 63, 59, 43, 43, 46, 54, 69, 55, 75, 55, 106, 95, 106, 150, 109, 134, 83] 2352
[152, 67, 37, 34, 24, 27, 57, 76, 50, 19, 16, 25, 28, 38, 15, 23, 18, 18, 21, 15, 40, 19, 27, 24, 22, 25, 42, 43, 54, 39, 24] 1119


In [12]:
# count order number by fdc and rdc per day: {date: (total #, fdc #, rdc #)}
fdc_rdc = {}
fdc_rdc_day_num = []
for i in range(len(day_list)):
     fdc_rdc_day_num.append(fdc_day_num[i] + rdc_day_num[i])
     fdc_rdc[day_list[i]] = (fdc_day_num[i] + rdc_day_num[i], fdc_day_num[i], rdc_day_num[i])
print("FDC", fdc, ", RDC", rdc, "---", fdc_rdc)

FDC 43 , RDC 2 --- {'2018-03-01': (341, 189, 152), '2018-03-02': (107, 40, 67), '2018-03-03': (99, 62, 37), '2018-03-04': (89, 55, 34), '2018-03-05': (78, 54, 24), '2018-03-06': (81, 54, 27), '2018-03-07': (133, 76, 57), '2018-03-08': (178, 102, 76), '2018-03-09': (112, 62, 50), '2018-03-10': (55, 36, 19), '2018-03-11': (58, 42, 16), '2018-03-12': (94, 69, 25), '2018-03-13': (121, 93, 28), '2018-03-14': (111, 73, 38), '2018-03-15': (78, 63, 15), '2018-03-16': (82, 59, 23), '2018-03-17': (61, 43, 18), '2018-03-18': (61, 43, 18), '2018-03-19': (67, 46, 21), '2018-03-20': (69, 54, 15), '2018-03-21': (109, 69, 40), '2018-03-22': (74, 55, 19), '2018-03-23': (102, 75, 27), '2018-03-24': (79, 55, 24), '2018-03-25': (128, 106, 22), '2018-03-26': (120, 95, 25), '2018-03-27': (148, 106, 42), '2018-03-28': (193, 150, 43), '2018-03-29': (163, 109, 54), '2018-03-30': (173, 134, 39), '2018-03-31': (107, 83, 24)}


In [13]:
print(sorted(fdc_rdc_day_num))
our_day_list = []
our_demand = {}
demand_num = 0
for i in range(len(day_list)):
     d = day_list[i]
     if 10000 >= fdc_rdc_day_num[i] >= 0:
          our_demand[d] = fdc_rdc_day_num[i]
          our_day_list.append(d)
          demand_num += fdc_rdc_day_num[i]
print(our_demand, len(our_demand))
print(demand_num, demand_num/len(our_day_list))

[55, 58, 61, 61, 67, 69, 74, 78, 78, 79, 81, 82, 89, 94, 99, 102, 107, 107, 109, 111, 112, 120, 121, 128, 133, 148, 163, 173, 178, 193, 341]
{'2018-03-01': 341, '2018-03-02': 107, '2018-03-03': 99, '2018-03-04': 89, '2018-03-05': 78, '2018-03-06': 81, '2018-03-07': 133, '2018-03-08': 178, '2018-03-09': 112, '2018-03-10': 55, '2018-03-11': 58, '2018-03-12': 94, '2018-03-13': 121, '2018-03-14': 111, '2018-03-15': 78, '2018-03-16': 82, '2018-03-17': 61, '2018-03-18': 61, '2018-03-19': 67, '2018-03-20': 69, '2018-03-21': 109, '2018-03-22': 74, '2018-03-23': 102, '2018-03-24': 79, '2018-03-25': 128, '2018-03-26': 120, '2018-03-27': 148, '2018-03-28': 193, '2018-03-29': 163, '2018-03-30': 173, '2018-03-31': 107} 31
3471 111.96774193548387


### Data

In [14]:
order_our_date = pd.DataFrame(data=None)
for i in range(len(our_day_list)):
      temp_data = order.loc[our_day_list[i]]
      order_our_date = pd.concat([order_our_date, temp_data])
order_our_date = order_our_date.sort_index()
order_our_date.head()
para_input = {"sample_set": order_our_date, "day_set": our_day_list, "fdc": fdc, "rdc": rdc}
np.save('./pre_data/setting.npy', para_input)

### Setting Parameter

In [15]:
order_fdc = order_our_date[order_our_date['dc_ori'] == fdc]
order_rdc = order_our_date[order_our_date['dc_ori'] == rdc]

In [16]:
# inventory, SKU
# sku_dict = {i: SKU id}
sku_list = order_our_date['sku_ID'].unique().tolist()
print(len(sku_list))
sku_dict = {}
for i in range(len(sku_list)):
     sku_dict[i] = sku_list[i]
print(sku_dict)

207
{0: '623d0a582a', 1: '38d636d2a6', 2: '6717b7c979', 3: '767ac490ed', 4: '068f4481b3', 5: 'd3e31fdd6e', 6: '427bea8ec7', 7: '7e4cb4952a', 8: 'a0e49f9966', 9: '8da781e2bd', 10: '5ae1bb1c76', 11: 'c83b8aa9b4', 12: 'b0d2964084', 13: 'fc5289b139', 14: '945094e307', 15: 'd47c6ca631', 16: '915cd3c4c0', 17: 'fc728c8ac1', 18: '0bfcedf96d', 19: 'eb3f2d2fd8', 20: '5f58bfd286', 21: '6420ea9ac2', 22: '4b65db43ac', 23: '904fbf8b97', 24: '12797fae91', 25: '589c2b865b', 26: '66bd21ebf1', 27: '93f9bac4b8', 28: '2efd0bcff7', 29: '1b3f5d4d21', 30: '8038a0e904', 31: '8d5f73c188', 32: '3c79df1d80', 33: 'b9f08a2a2a', 34: 'f255daf3ab', 35: '2523d051fd', 36: 'c98d32ff09', 37: '7c3627ba7c', 38: 'c4ea4afceb', 39: 'ad71b48a82', 40: 'f363f97793', 41: 'f52ec33325', 42: 'eaa2524751', 43: 'c56544784c', 44: '2875387e96', 45: '8ad5789d74', 46: '9400971aad', 47: '8dc4a01dec', 48: '83e24b70d5', 49: '9e452b5aee', 50: 'fbce41fd82', 51: 'b10b452ea9', 52: '5eced3cd82', 53: '93301ecfed', 54: 'c5f6b945e7', 55: '6da843324a

In [17]:
# inventory in FDC per day
# sku_inv_fdc = {i: # in fdc}
# E.g., the first day
order_fdc_fdc_the_day = order_fdc.loc[our_day_list[0]]
sku_inv_fdc = {}
for i in range(len(sku_list)):
     tmp_sku = sku_list[i]
     tmp_sku_order = order_fdc_fdc_the_day[order_fdc_fdc_the_day['sku_ID'] == tmp_sku]
     sku_inv_fdc[i] = tmp_sku_order['quantity'].sum()
print(our_day_list[0], sku_inv_fdc)

2018-03-01 {0: 137, 1: 0, 2: 0, 3: 2, 4: 4, 5: 0, 6: 0, 7: 9, 8: 0, 9: 0, 10: 7, 11: 3, 12: 2, 13: 3, 14: 1, 15: 3, 16: 0, 17: 1, 18: 0, 19: 0, 20: 1, 21: 0, 22: 0, 23: 5, 24: 0, 25: 2, 26: 0, 27: 2, 28: 0, 29: 0, 30: 0, 31: 2, 32: 7, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 3, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 3, 48: 0, 49: 2, 50: 0, 51: 3, 52: 0, 53: 1, 54: 2, 55: 0, 56: 0, 57: 3, 58: 0, 59: 0, 60: 0, 61: 2, 62: 0, 63: 0, 64: 3, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 1, 71: 0, 72: 0, 73: 0, 74: 1, 75: 0, 76: 0, 77: 1, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 

In [18]:
# total inventory in FDC
# sku_inv_fdc = {i: total # in fdc}
sku_inv_fdc = {}
for i in range(len(sku_list)):
     tmp_sku = sku_list[i]
     tmp_sku_order = order_fdc[order_fdc['sku_ID'] == tmp_sku]
     sku_inv_fdc[i] = tmp_sku_order['quantity'].sum()
print(sku_inv_fdc)

{0: 244, 1: 98, 2: 20, 3: 43, 4: 315, 5: 3, 6: 9, 7: 153, 8: 0, 9: 8, 10: 23, 11: 12, 12: 8, 13: 7, 14: 2, 15: 93, 16: 19, 17: 6, 18: 6, 19: 16, 20: 32, 21: 0, 22: 78, 23: 56, 24: 0, 25: 39, 26: 22, 27: 17, 28: 0, 29: 0, 30: 0, 31: 29, 32: 206, 33: 0, 34: 35, 35: 13, 36: 10, 37: 0, 38: 46, 39: 1, 40: 0, 41: 0, 42: 2, 43: 3, 44: 0, 45: 0, 46: 0, 47: 122, 48: 7, 49: 85, 50: 94, 51: 13, 52: 36, 53: 20, 54: 9, 55: 0, 56: 0, 57: 70, 58: 3, 59: 26, 60: 0, 61: 4, 62: 0, 63: 0, 64: 165, 65: 0, 66: 2, 67: 0, 68: 0, 69: 19, 70: 32, 71: 0, 72: 22, 73: 0, 74: 2, 75: 0, 76: 0, 77: 2, 78: 0, 79: 13, 80: 5, 81: 15, 82: 0, 83: 7, 84: 0, 85: 0, 86: 0, 87: 10, 88: 13, 89: 23, 90: 48, 91: 3, 92: 2, 93: 4, 94: 0, 95: 0, 96: 1, 97: 0, 98: 1, 99: 0, 100: 1, 101: 0, 102: 0, 103: 1, 104: 19, 105: 0, 106: 31, 107: 0, 108: 8, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 1, 116: 0, 117: 1, 118: 1, 119: 2, 120: 0, 121: 0, 122: 0, 123: 22, 124: 0, 125: 0, 126: 0, 127: 2, 128: 2, 129: 0, 130: 0, 131: 28, 13

In [19]:
# demand
order_id_list = order_our_date['order_ID'].unique().tolist()
print(len(order_id_list))

3471


In [20]:
# demand
# order_dict = {j: {sku key: #}}
order_dict = {}
# demand_order = {j: demand #}
demand_order = {}
index_j = 0
new_data = pd.DataFrame(data=None)

for i in order_id_list:
    tmp = order_our_date[order_our_date['order_ID'] == i]
    tmp_list = tmp['sku_ID'].values.tolist()
    tmp_list2 = [sku_list.index(j) for j in tmp_list]
    tmp_list3 = sorted(tmp_list2)
    tmp_list3 = list(dict.fromkeys(tmp_list3))

    t = tmp[tmp['sku_ID'] == tmp_list[0]].index.tolist()

    tmp_value = {}
    for j in range(0, len(tmp_list3)):
         tmp_value[tmp_list3[j]] = int(tmp[tmp['sku_ID'] == sku_list[tmp_list3[j]]]['quantity'])
    if tmp_value in order_dict.values():
         key_list = list(order_dict.keys())
         value_list = list(order_dict.values())
         tmp_key = key_list[value_list.index(tmp_value)]
         demand_order[tmp_key] += 1
         tmp_data = pd.DataFrame(data=[[tmp_key, t[0]]], columns=['order_type', 'order_time'])
         new_data = pd.concat([new_data, tmp_data])
    else:
         order_dict[index_j] = tmp_value
         demand_order[index_j] = 1
         tmp_data = pd.DataFrame(data=[[index_j, t[0]]], columns=['order_type', 'order_time'])
         new_data = pd.concat([new_data, tmp_data])
         index_j += 1
print(len(order_dict))
print(order_dict)
print(demand_order)

503
{0: {0: 1}, 1: {1: 1, 2: 1}, 2: {3: 1}, 3: {4: 1, 5: 2}, 4: {2: 1, 6: 1}, 5: {7: 1}, 6: {8: 1}, 7: {9: 1}, 8: {10: 1}, 9: {11: 2}, 10: {12: 1}, 11: {13: 1}, 12: {14: 1, 15: 1}, 13: {16: 1}, 14: {17: 1, 18: 1}, 15: {19: 1, 20: 1}, 16: {21: 2}, 17: {22: 2}, 18: {20: 1, 23: 2}, 19: {24: 1}, 20: {25: 10}, 21: {26: 1}, 22: {25: 2, 27: 1}, 23: {25: 1}, 24: {28: 1}, 25: {29: 2}, 26: {27: 1}, 27: {0: 4}, 28: {30: 1}, 29: {0: 3}, 30: {15: 1}, 31: {31: 1}, 32: {32: 1}, 33: {17: 3}, 34: {5: 2, 33: 2}, 35: {34: 1}, 36: {15: 1, 27: 1}, 37: {5: 1, 35: 1}, 38: {36: 3, 37: 1}, 39: {23: 1}, 40: {38: 1}, 41: {31: 1, 39: 1}, 42: {5: 1, 33: 1}, 43: {20: 2}, 44: {5: 1, 40: 1}, 45: {4: 2, 5: 4}, 46: {41: 2}, 47: {42: 3}, 48: {5: 1, 43: 1}, 49: {17: 1}, 50: {15: 3}, 51: {44: 1}, 52: {39: 1}, 53: {0: 1, 11: 1}, 54: {45: 1}, 55: {46: 1}, 56: {29: 1, 41: 1}, 57: {29: 1}, 58: {0: 2}, 59: {47: 1}, 60: {48: 1}, 61: {49: 2}, 62: {5: 2, 50: 1}, 63: {51: 1}, 64: {7: 2, 52: 2}, 65: {32: 2}, 66: {53: 1}, 67: {54: 2

In [21]:
a = [1, 2, 3, 4, 5]
c = [0, 1, 3]
b = [a[i] for i in c]
print(b)

[1, 2, 4]
